# Chose final matches for calibration
Could implement your own post processing that accepts the best match across multiple match settings.

Use `notebooks/Map_Match_QAQC.ipynb` to inspect matches

In [1]:
import pickle
import geopandas as gpd
import pandas as pd
from tqdm import tqdm
from shapely.ops import MultiLineString, LineString
import geopandas as gpd

from bikewaysim.paths import config
from bikewaysim.impedance_calibration import speedfactor, stochastic_optimization
from bikewaysim.map_matching import map_match
from bikewaysim.network import prepare_network, modeling_turns
from bikewaysim.routing import rustworkx_routing_funcs
from bikewaysim.map_matching import post_process

In [3]:
# print the available match dicts
print([x.stem for x in config['matching_fp'].glob('match_dict_full_*.pkl')])
matching_index = 0  # Change this to the index of the match dict you want to use
# matching_index = 5


['match_dict_full_0']


In [4]:
with (config['matching_fp'] / f'match_dict_full_{matching_index}.pkl').open('rb') as fh:
    match_dict = pickle.load(fh)

cutoff = 0.90 # set pct of points that need to be matched
above_threshold, below_threshold, failed_matches, match_ratios = post_process.mapmatch_results(match_dict,cutoff)
match_dict = {key:item for key,item in match_dict.items() if key in above_threshold}

597 / 682 (88%) successful matches
81 / 682 (12%) partial matches
4 / 682 (1%) failed matches


In [27]:
with (config['calibration_fp'] / 'final_matches.pkl').open('wb') as fh:
    pickle.dump(match_dict,fh)

## Choose how to group matches for calibration
For this step we use CycleAtlanta data. These cells would need to be modified based on the data source.

In [ ]:
# Cycle Atlanta trip and user data
trips = pd.read_pickle(config['cycleatl_fp']/'trips_4.pkl')
trips = trips.loc[trips['tripid'].isin(list(match_dict.keys()))]
users = pd.read_pickle(config['cycleatl_fp']/'users_4.pkl')
users = users[users['userid'].isin(set(trips['userid'].tolist()))]
print(trips.shape[0],'trips')
print(users.shape[0],'users')

597 trips
259 users


In [24]:
trips_for_calibration = [] # group_name, [trip_ids]

In [ ]:
# groups by user for user specific calibration
user_groups = [(userid,list(trip_ids)) for userid, trip_ids in trips.groupby('userid')['tripid'].unique().reset_index().values]

In [ ]:
# selects a random trip from each user
import random
random.seed(2)
grouped = trips.groupby('userid')['tripid'].apply(lambda x: random.choice(list(x)))
random_trips = [('random_trips',grouped.tolist())]

In [ ]:
# selects a random trip from each user, but groups the trips
# by user stated rider type (could also do education level, age, etc.)
import random
random.seed(2)
grouped = trips.groupby('userid')['tripid'].apply(lambda x: random.choice(list(x)))
not_fearless_users = users.loc[users['rider_type']!='Strong & fearless','userid'].tolist()
not_fearless = [tripid for userid, tripid in grouped.values if userid in not_fearless_users]
fearless = [tripid for userid, tripid in grouped.values if userid not in not_fearless_users]
not_fearless = [('notfearless',not_fearless)]
fearless = [('fearless',fearless)]

In [ ]:
trips_for_calibration += user_groups + random_trips + not_fearless + fearless

In [26]:
with (config['calibration_fp'] / 'subsets.pkl').open('wb') as fh:
    pickle.dump(trips_for_calibration,fh)